# Ergast API Data Ingestion

## Imports

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime
import asyncio

module_path = os.path.abspath(os.path.join("..", "scripts"))
if module_path not in sys.path:
    sys.path.append(module_path)

from f1_data_fetcher import *

In [2]:
START_YEAR = 1995
END_YEAR = datetime.now().year

## API Requests

### Race Data

In [12]:
l = ["Bear", "ANT", "Antelope", "CHICKEN", "Brachiosaurus", "WHALE", "Chipmunk", "SEAL", "Penguin"]
l = [x.lower() for x in l]

ordered_l = sorted(l, key=lambda x: len(x))
for i in ordered_l:
    if len(i) > 5:
        print(i.upper(), "(italicized)")
    elif len(i) <= 5:
        print(i.upper(), "(bold)")

ANT (bold)
BEAR (bold)
SEAL (bold)
WHALE (bold)
CHICKEN (italicized)
PENGUIN (italicized)
ANTELOPE (italicized)
CHIPMUNK (italicized)
BRACHIOSAURUS (italicized)


In [13]:
numbers = [1, 3, 5, 9, 35, 83]
sorted_numbers = sorted([num for num in numbers if num % 2 != 0])
print(sorted_numbers)

[1, 3, 5, 9, 35, 83]


In [15]:
current_year = 2022  # current year
birth_year = current_year - (
    100 - 27
)  # calculate birth year based on current year and age
birth_year

1949

In [3]:
# Race info (circuit, date, country)
race_data_df = fetch_race_info(START_YEAR, END_YEAR)
race_data_df.head()

race_data_df.to_csv(
    f"../data/raw/Race_Information_{START_YEAR}_{END_YEAR}.csv", index=False
)

In [4]:
# Results
result_df = asyncio.run(fetch_race_results())

2024-07-03 11:53:09,665 - INFO - Fetching race data for Season: 1995, Round: 1
2024-07-03 11:53:17,623 - INFO - Fetching race data for Season: 1995, Round: 1
2024-07-03 11:53:25,291 - INFO - Fetching race data for Season: 1995, Round: 1
2024-07-03 11:53:33,021 - INFO - Fetching race data for Season: 1995, Round: 1
2024-07-03 11:53:44,687 - INFO - Fetching race data for Season: 1995, Round: 1


KeyboardInterrupt: 

In [ ]:
race_results_data_df_list = fetch_all_races(START_YEAR, END_YEAR)
race_results_data_df = pd.concat(race_results_data_df_list, ignore_index=True)
race_results_data_df.to_csv(
    f"../data/raw/Race_Results_{START_YEAR}_{END_YEAR}.csv", index=False
)

In [ ]:
# Fetch all F1 drivers
all_drivers_df = fetch_all_f1_drivers()
all_drivers_df.to_csv("../data/raw/drivers_information.csv", index=False)

In [ ]:
# Fetch all F1 circuits
all_circuits_df = fetch_all_f1_circuits()
all_circuits_df.to_csv("../data/raw/circuits_information.csv", index=False)

In [ ]:
# Fetch driver and constructor standings
driver_standing_df = fetch_all_driver_standings(race_data_df)
driver_standing_df.to_csv("../data/raw/driver_standings.csv", index=False)